# <b> Sprawozdanie PSD - lista 2 </b> #
### Stefan Borek (260359) ###
4-5-2022

## Polecania: ##
1. Pobrać listę słów, które będą uznawane za poprawne oraz zbiór danych tekstowych (tzw. korpus) dla języka angielskiego.
2. Wykorzystać podejścia naiwne oraz filtr Blooma (o rozmiarze równym liczbie słów) do oznaczenia słów z korpusu niewystępujących w liście słów (tj. niepoprawnych).
3. Porównać wymagania pamięciowe oraz czasowe dla podejść naiwnych oraz filtru Blooma.
4. Przedstawić listę 10 słów najczęściej oznaczanych jako niepoprawne, oraz wyznaczyć po 3 najbliższe propozycje z listy poprawnych słów na podstawie odległości Levenshteina 4.

In [1]:
import nltk
from nltk.corpus import words, movie_reviews
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from collections import Counter
import time
import re

In [3]:
# wczytanie do listy - words (wszystkie słowa)
all_words = words.words('en')
len(all_words)

235886

In [4]:
# wczytanie do listy
word_list = list(movie_reviews.words())
len(word_list)

1583820

Usuniecie niepotrzebnych znaków.

In [6]:
movie_reviews_words = []
for w in word_list:
    if re.match(r'^[a-zA-Z]+$', w):
        movie_reviews_words.append(w)
        
len(movie_reviews_words)

1329753

## Implementacja filtru Blooma ##
Filtr blooma to struktura danych zaprojektowana do sprawdzania, czy szukany element znajduje się w zdefiniowanym zbiorze elementów. Często jest wykorzystywany np. przed rejestracją użytkownika na portalach internetowych bay sprawdzić czy nazwa użytkonika jest już zjęta.

Użycie biblioteki <b>bloom-filter2</b>

In [9]:
from bloom_filter2 import BloomFilter

In [10]:
bloom = BloomFilter(max_elements=len(all_words), error_rate=0.05)
bloom

BloomFilter(ideal_num_elements_n=235886, error_rate_p=0.050000, num_bits_m=1470803)

In [11]:
for item in all_words:
    bloom.add(item)

In [12]:
accuracy = 0
errors = []

start = time.time()
for item in movie_reviews_words:
    if item not in bloom:
        errors.append(item)
    else:
        accuracy += 1
end = time.time()
print('Czas filtr blooma:')
print(end-start, 's')

Czas filtr blooma:
8.961230993270874 s


In [13]:
accuracy /= len(word_list)
print('errors: ', len(errors))
print('accuracy: ', accuracy)

errors:  181253
accuracy:  0.7251455342147466


In [14]:
error_words = Counter(errors)

In [15]:
error_words = dict(sorted(error_words.items(), key=lambda item: item[1], reverse=True))

In [16]:
top_mistakes = list(error_words)[:10]
top_mistakes

['has',
 'characters',
 'films',
 'doesn',
 'scenes',
 'movies',
 'seems',
 'makes',
 'isn',
 'things']

Wyszukanie najbliższych słów poprzez odległość <b>Levenshteina</b> obliczam ją przy pomocy funkcji edit_distance z biblioteki nltk.\
<b>Odległość Levenshteina</b> - ilość znaków w słowie, które powinny być zmienione aby otrzymać oczekiwane słowo.

In [18]:
print('Wyniki:')
for w1 in top_mistakes:
    min_word = ''
    min_dist = len(w1)
    for w2 in all_words:
        dd = nltk.edit_distance(w1, w2)
        if dd < min_dist:
            min_dist = dd
            min_word = w2
    print(w1, '-->', min_word)

Wyniki:
has --> as
characters --> character
films --> film
doesn --> does
scenes --> scene
movies --> movie
seems --> seem
makes --> jakes
isn --> hisn
things --> thing


### Wnioski: ###
Najsześciej występujące niepoprawne słowa są związane z ich odmianą. W korpusie nie znalazły sę odmiany oraz liczby mnogie niektórych słów z czy dość dobrze poradziła sobie nasza selekcja z użyciem odległości Levenshteina.

### Podejscie naiwne ###
Przechodzimy po koleji po słowach do znalezienia i szukamy ich w korpusie.

In [20]:
import multiprocessing as mp
import time

In [21]:
def naive_aproach(words_list, corpus):
    mistakes = 0
    for word in words_list:
        if word not in corpus:
            mistakes += 1

    return mistakes


if __name__ == '__main__':
    with mp.Pool(processes=7) as pool:
        start = time.time()
        p = mp.Process(target=naive_aproach, args=(word_list[:50000], all_words))
        p.run()

    
    end = time.time()
    print("Czas podejście naiwne:")
    print(end-start, 's')

Czas podejście naiwne:
85.86287879943848 s


## Wnioski ##
Podejście naiwne dla 50000 słów zajmuje już ponad 85 s co daje czas nieporównywalnie większy od filtru Blooma to pokazuje jaką przewagę daje nam zastosowanie tej metody i jak duże ma ona możliwości. 